In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

# =============================================================================
# CONFIGURATION
# =============================================================================
INPUT_FILE = 'ilr_coordinates.csv'
OUTPUT_FILE = 'final_cluster_labels.csv'
PLOT_FILE = 'BIC_SCORE_PLOT.png'

# The 4 Coordinates we calculated in Phase 2
FEATURES = ['ilr_Activity', 'ilr_Profitability', 'ilr_Solvency', 'ilr_Liquidity_Texture']

print("--- STARTING PHASE 3: BEHAVIORAL CLUSTERING (GMM) ---")

# 1. LOAD DATA
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"Loaded Data: {df.shape[0]} rows.")
except FileNotFoundError:
    print(f"CRITICAL ERROR: Could not find '{INPUT_FILE}'. Run Phase 2 first.")
    exit()

# 2. PREPARE DATA
# We filter for the features. We drop any remaining NaNs just in case (should be 0).
X = df[FEATURES].dropna()
companies_aligned = df.loc[X.index] # Keep metadata aligned

# Standardize? 
# While CoDa is already normalized, scaling helps GMM converge faster.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# =============================================================================
# STEP A: FIND OPTIMAL CLUSTERS (BIC Score)
# =============================================================================
print("Calculating Optimal Number of Clusters (BIC)...")
# We test K=2 to K=20. The lower the BIC, the better the model fits the data.

bic_scores = []
k_range = range(2, 21)

best_bic = np.inf
best_k = 0

for k in k_range:
    # covariance_type='full' allows clusters to have different shapes (ellipses)
    gmm = GaussianMixture(n_components=k, covariance_type='full', random_state=42, n_init=3)
    gmm.fit(X_scaled)
    
    bic = gmm.bic(X_scaled)
    bic_scores.append(bic)
    
    # Track best model
    if bic < best_bic:
        best_bic = bic
        best_k = k
    
    print(f"   K={k}: BIC={bic:.0f}")

print(f"\n-> OPTIMAL K FOUND: {best_k} Clusters (Lowest BIC)")

# Plot BIC for Thesis
plt.figure(figsize=(10, 6))
plt.plot(k_range, bic_scores, marker='o', linestyle='-', color='b')
plt.title('Optimal Cluster Selection (Bayesian Information Criterion)')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('BIC Score (Lower is Better)')
plt.grid(True, alpha=0.3)
plt.axvline(best_k, color='r', linestyle='--', label=f'Optimal K={best_k}')
plt.legend()
plt.savefig(PLOT_FILE)
print(f"-> BIC Plot saved to '{PLOT_FILE}'")

# =============================================================================
# STEP B: FIT FINAL MODEL
# =============================================================================
print(f"Fitting Final GMM Model with K={best_k}...")

final_gmm = GaussianMixture(n_components=best_k, covariance_type='full', random_state=42, n_init=10)
final_gmm.fit(X_scaled)

# 1. Hard Assignment (The specific cluster ID)
cluster_labels = final_gmm.predict(X_scaled)

# 2. Soft Probability (For Reliance-style "Hybrid" Analysis)
# How confident is the model? (Max probability)
probs = final_gmm.predict_proba(X_scaled)
max_probs = probs.max(axis=1)

# =============================================================================
# SAVE OUTPUT
# =============================================================================
# Attach results back to the original dataframe
# Note: We use .loc to ensure we match the indices correctly
output_df = df.copy()
output_df['Cluster_ID'] = np.nan
output_df['Cluster_Prob'] = np.nan

output_df.loc[X.index, 'Cluster_ID'] = cluster_labels
output_df.loc[X.index, 'Cluster_Prob'] = max_probs

# Save
output_df.to_csv(OUTPUT_FILE, index=False)

print("-" * 50)
print("SUCCESS! Phase 3 Complete.")
print(f"Generated {best_k} Behavioral Peer Groups.")
print(f"Saved Clustered Data to: {OUTPUT_FILE}")
print("-" * 50)

# Optional: Print Cluster Stats
print("\nCluster Distribution:")
print(output_df['Cluster_ID'].value_counts().sort_index())